In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(["science", "notebook", "grid", "dark_background"])

In [32]:
class CubicSpline:
    def __init__(self, x: np.ndarray, y: np.ndarray, target: np.ndarray) -> None:
        """ Do a cubic spline interpolation.
        Args:
            - x: Array of x coordinates of the data points.
            - y: Array of y coordinates of the data points.
            - target: Points at which to evaluate the spline.
        Returns:
            - The evaluated spline values."""
        self.x : np.ndarray = x # x coordinates of the data points
        self.y : np.ndarray = y # y coordinates of the data points
        self.target = target # x points at which to evaluate the spline

    def calculate_spline_coefficients(self):
        """
        Calculate the coefficients of cubic splines for given data points.

        Parameters:
        - None (data points are stored in the object)
        
        Returns:
        - A matrix of spline coefficients.
        """
        n = len(self.x) - 1
        h = np.diff(self.x) 

        # Calculate the matrix A
        A = np.zeros((n + 1, n + 1))
        A[0, 0] = 1
        A[n, n] = 1
        for i in range(1, n):
            A[i, i - 1] = h[i - 1]
            A[i, i] = 2 * (h[i - 1] + h[i])
            A[i, i + 1] = h[i]

        # Calculate the vector b
        b = np.zeros(n + 1)
        for i in range(1, n):
            b[i] = 3 * ((self.y[i + 1] - self.y[i]) / h[i] - (self.y[i] - self.y[i - 1]) / h[i - 1])

        # Solve the system of linear equations
        m = np.linalg.solve(A, b)

        # Calculate coefficients of the splines
        S = np.zeros((n, 4))
        for i in range(n):
            S[i, 0] = self.y[i]
            S[i, 1] = (self.y[i + 1] - self.y[i]) / h[i] - h[i] * (2 * m[i] + m[i + 1]) / 3
            S[i, 2] = m[i]
            S[i, 3] = (m[i + 1] - m[i]) / (3 * h[i])

        return S
    
    def evaluate_spline(self):
        """
        Evaluate the spline at given points xi.

        Parameters:
        - None (data points and target points are stored in the object)

        Returns:
        - The evaluated spline values.
        """
        n = len(self.x) - 1
        result = np.zeros_like(self.target)
        coeffs = self.calculate_spline_coefficients()

        for j, xj in enumerate(self.target):
            for i in range(n):
                if xj >= self.x[i] and xj <= self.x[i + 1]:
                    dx = xj - self.x[i]
                    result[j] = (coeffs[i, 0] + coeffs[i, 1] * dx + 
                                 coeffs[i, 2] * dx**2 + coeffs[i, 3] * dx**3)
                    break
                
        return result
    
    def plot_spline(self) -> None:
        """
        Plot the spline and the data points.

        Parameters:
        - plotting_points: Points at which to evaluate the spline.

        Returns:
        - None (the plot is shown)
        """
        result = self.evaluate_spline()
        plt.plot(self.x, self.y, 'o', label='Data points')
        plt.plot(self.target, result, label='Cubic spline')
        plt.legend()

        return None

In [33]:
x = np.array([1, 2, 3, 4, 5])
y = np.array([30, 15, 32, 18, 25])

target = np.array([3.5])

spline = CubicSpline(x, y, target)

result = spline.evaluate_spline()
coeffs = spline.calculate_spline_coefficients()

print(coeffs)
print(f" The spline values are: {result}")

[[ 30.         -26.16071429   0.          11.16071429]
 [ 15.           7.32142857  33.48214286 -23.80357143]
 [ 32.           2.875      -37.92857143  21.05357143]
 [ 18.          -9.82142857  25.23214286  -8.41071429]]
 The spline values are: [26.58705357]
